In [ ]:
#@title GPU information
!nvidia-smi

In [ ]:
#@title Google Drive Mount
PasteAll_link = "https://pasteall.org/media/9/a/9a2231663cff06e4536f9b888e985b4f.blend" #@param {type : "string"}
file_name = "60_F40" #@param {type : "string"}
file_name += ".blend"

if file_name not in os.listdir("."):
  !wget $PasteAll_link

  name = PasteAll_link.split("/")[-1]
  print("\nDownloaded:")
  print(file_name)
  !mv $name $file_name


In [ ]:
#@title ## Download && Install Blender on VM
blender_version = "blender2.92.0" #@param ["blender2.92.0"]

if blender_version == "blender2.92.0":
    # download_path="https://download.blender.org/release/Blender2.91/blender-2.91.2-linux64.tar.xz"
    download_path="https://download.blender.org/release/Blender2.92/blender-2.92.0-linux64.tar.xz"

!mkdir $blender_version
if blender_version == "blender2.92.0":
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1

import os

#  This Block is required as some weird behaviors with libtcmalloc appeared in the colab VM
os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

# Unpack and move blender
!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

In [78]:
#@title Setup Info { run: "auto", vertical-output: true, form-width: "50%" }
#@markdown Please configure your setup
Samples_Count = 128 #@param {type:"integer"}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}


#blender_version = "blender2.91.0" #@param ["blender2.91.0"]
#file_name = '60_F40'  #@param {type: "string"}
file_name = file_name.split(".")[0]
#@markdown ---
#@markdown Single Frame render
single_frame = True #@param {type:"boolean"}
frame =  6#@param {type:"integer"}
#@markdown ---
#@markdown Sequence render
sequence =  False #@param {type:"boolean"}
start_frame =   1#@param {type:"integer"}
end_frame =   5#@param {type:"integer"}

data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "bpy.context.scene.cycles.samples = "+str(Samples_Count)+"\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for scene in bpy.data.scenes:\n"+\
    "    scene.render.tile_x = 256\n"+\
    "    scene.render.tile_y = 256\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)


path_to_blend = './' + file_name + '.blend'
output_path = './output_' + file_name + '/output####.png'


In [ ]:
#@title Rendering
if sequence:
  !sudo ./$blender_version/blender -P './setgpu.py'  -b '{path_to_blend}' -E CYCLES -o '{output_path}' -s {start_frame} -e {end_frame} -a
else:
  !sudo ./$blender_version/blender -P 'setgpu.py' -b '{path_to_blend}' -P 'setgpu.py' -o '{output_path}' -f {frame}